# 完整脚本
       
       就三步，read,compile,forward

In [29]:
from openvino.runtime import Core
import numpy as np
import cv2 as cv
from PIL import Image
import matplotlib.pyplot as plt
import openvino as ov

In [2]:
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)

from notebook_utils import download_file

In [3]:
#看你选择什么模型
ir_model_url = 'https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/'
ir_model_name_xml = 'classification.xml'
ir_model_name_bin = 'classification.bin'

download_file(ir_model_url + ir_model_name_xml, filename=ir_model_name_xml, directory='model')
download_file(ir_model_url + ir_model_name_bin, filename=ir_model_name_bin, directory='model')

model\classification.xml:   0%|          | 0.00/179k [00:00<?, ?B/s]

model\classification.bin:   0%|          | 0.00/4.84M [00:00<?, ?B/s]

WindowsPath('D:/myjupyter/深度学习/deployment/openvino/open1/model/classification.bin')

In [5]:
ie = Core()
devices = 'CPU'
model_xml = 'model/classification.xml'
#model_bin = 'resnet18.bin'
#根本不需要使用.xml文件，不论是onnx，tensorflow,还是paddlepaddle，都可以直接读取，不用再MO处理
net = ie.read_model(model= model_xml,
                    #weights = model_bin
                     )

#如果是pytorch模型，需要转化以下
#example_input = torch.zeros((1, 3, 224, 224))
#net= ov.convert_model(pt_model, example_input=example_input)

compiled_model = ie.compile_model(model=net,device_name="CPU")
#compiled_model = core.compile_model(model, device, {"PERFORMANCE_HINT": "LATENCY"})
#compiled_model = core.compile_model(model, device, {"PERFORMANCE_HINT": "THROUGHPUT"})
#输入输出层，每个层有名字和形状，查询后数据填充把
input_layer =compiled_model.inputs[0]
out_layer =compiled_model.outputs[0]


In [ ]:
#保存导出，转换格式
ov.save_model(model_onnx, output_model="model/exported_onnx_model.xml")

# 准备输入

In [ ]:
import cv2
image_filename = "data/coco_hollywood.jpg"
image_filename = download_file(
    "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/image/coco_hollywood.jpg",
    directory="data")

In [17]:
image = cv2.imread("data/coco_hollywood.jpg")
N, C, H, W = input_layer.shape
resized_image = cv2.resize(src=image, dsize=(W, H))
resized_image.shape
#normalized_image = normalize(resized_image)
input_data = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0).astype(np.float32)
input_data.shape
#这些信息一般会和模型放在一起，去模型文件夹中可以找到。同时也有输出结果的信息，解释每一个维度
#image = np.float32(image)/255.0
#image[:,:,]-= (np.float32(0.485),np.float32(0.456),np.float32(0.406))
#image = image.transpose(2,0,1)
result = compiled_model([input_data])[out_layer]

# 特殊形状

In [ ]:
from openvino.runtime import Core, PartialShape

ie = Core()
segmentation_model_xml = "model/segmentation.xml"
segmentation_model = ie.read_model(model=segmentation_model_xml)
segmentation_input_layer = segmentation_model.inputs[0]
segmentation_output_layer = segmentation_model.outputs[0]

new_shape = PartialShape([1, 3, 544, 544])#新的网络大小，batch大小也可以修改

#这里返回None
segmentation_model.reshape({segmentation_input_layer.any_name: new_shape})
segmentation_compiled_model = ie.compile_model(model=segmentation_model, device_name="CPU")

# 视频异步对象检测

In [ ]:
import notebook_utils as utils
def draw_boxes(frame, boxes):
    for label, score, box in boxes:
        # choose color for the label
        color = tuple(map(int, colors[label]))
        # draw box
        cv2.rectangle(img=frame, pt1=box[:2], pt2=box[2:], color=color, thickness=3)
        # draw label name inside the box
        cv2.putText(img=frame, 
                    text=f"{classes[label]} {score:.2f}", 
                    org=(box[0] + 10, box[1] + 30),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX, 
                    fontScale=frame.shape[1] / 1000, 
                    color=color,
                    thickness=1, 
                    lineType=cv2.LINE_AA)

    return frame


# main processing function to run object detection
def run_object_detection(source=0, flip=False, use_popup=False, skip_first_frames=0):
    player = None
    try:
        # create video player to play with target fps
        player = utils.VideoPlayer(source=source, flip=flip, fps=30, skip_first_frames=skip_first_frames)
        # start capturing
        player.start()
        if use_popup:
            title = "Press ESC to Exit"
            cv2.namedWindow(winname=title, flags=cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)

        processing_times = collections.deque()
        request = compiled_model.create_infer_request()
        while True:
            # grab the frame
            frame = player.next()
            if frame is None:
                print("Source ended")
                break
            # if frame larger than full HD, reduce size to improve the performance
            scale = 1280 / max(frame.shape)
            if scale < 1:
                frame = cv2.resize(src=frame, dsize=None, fx=scale, fy=scale,
                                   interpolation=cv2.INTER_AREA)

            # resize image and change dims to fit neural network input
            input_img = cv2.resize(src=frame, dsize=(width, height), interpolation=cv2.INTER_AREA)
            # create batch of images (size = 1)
            input_img = input_img[np.newaxis, ...]

            # measure processing time

            start_time = time.time()
            # get results
            request.infer(inputs={input_layer.any_name: input_img})
            results = request.get_output_tensor(output_layer.index).data
            stop_time = time.time()
            # get poses from network results
            #解码输出
            boxes = process_results(frame=frame, results=results)

            # draw boxes on a frame显示结果函数
            frame = draw_boxes(frame=frame, boxes=boxes)

            processing_times.append(stop_time - start_time)
            # use processing times from last 200 frames
            if len(processing_times) > 200:
                processing_times.popleft()

            _, f_width = frame.shape[:2]
            # mean processing time [ms]
            processing_time = np.mean(processing_times) * 1000
            fps = 1000 / processing_time
            cv2.putText(img=frame, text=f"Inference time: {processing_time:.1f}ms ({fps:.1f} FPS)", org=(20, 40),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=f_width / 1000,
                        color=(0, 0, 255), thickness=1, lineType=cv2.LINE_AA)

            # use this workaround if there is flickering
            if use_popup:
                cv2.imshow(winname=title, mat=frame)
                key = cv2.waitKey(1)
                # escape = 27
                if key == 27:
                    break
            else:
                # encode numpy array to jpg
                _, encoded_img = cv2.imencode(ext=".jpg", img=frame,
                                              params=[cv2.IMWRITE_JPEG_QUALITY, 100])
                # create IPython image
                i = display.Image(data=encoded_img)
                # display the image in this notebook
                display.clear_output(wait=True)
                display.display(i)
    # ctrl-c
    except KeyboardInterrupt:
        print("Interrupted")
    # any different error
    except RuntimeError as e:
        print(e)
    finally:
        if player is not None:
            # stop capturing
            player.stop()
        if use_popup:
            cv2.destroyAllWindows()

In [ ]:
run_object_detection(source=0, flip=True, use_popup=False)

#video_file = "../201-vision-monodepth/data/Coco Walking in Berkeley.mp4"
#run_object_detection(source=video_file, flip=False, use_popup=False)

# latency mode + shared memory
  可以提升计算性能,真的会变快
  

In [30]:
#模式设置

ov_auto_model = ie.compile_model(net, device_name="AUTO", 
                        config={"PERFORMANCE_HINT": "LATENCY"})
#将图片进行连续化处理
c_input_image = np.ascontiguousarray(input_data, dtype=np.float32)
#不用拷贝空间
input_tensor = ov.Tensor(c_input_image, shared_memory=True)

#result = ov_auto_model(input_tensor)[ov_auto_model.output(0)][0]

# 评估模型速度用到的函数

In [23]:
INFER_NUMBER = 10

import time
def benchmark_model(model, 
                    input_data: np.ndarray,
                    benchmark_name: str,
                    device_name: str = "CPU") -> float:
    # measure the first inference separately -  it may be slower as it contains also initialization
    start = time.perf_counter()
    model(input_data)
    end = time.perf_counter()
    first_infer_time = end - start
    print(f"{benchmark_name} on {device_name}. First inference time: {first_infer_time :.4f} seconds")

    # benchmarking
    start = time.perf_counter()
    for _ in range(INFER_NUMBER):
        model(input_data)
    end = time.perf_counter()

    # elapsed time
    infer_time = end - start

    # print second per image and FPS
    mean_infer_time = infer_time / INFER_NUMBER
    mean_fps = INFER_NUMBER / infer_time
    print(f"{benchmark_name} on {device_name}: {mean_infer_time :.4f} seconds per image ({mean_fps :.2f} FPS)")

    return mean_infer_time

In [24]:
benchmark_model(compiled_model,input_data,'name')

name on CPU. First inference time: 0.3412 seconds
name on CPU: 0.3432 seconds per image (2.91 FPS)


0.34319900260015856

In [31]:
benchmark_model(ov_auto_model,input_data,'ov_model')

ov_model on CPU. First inference time: 0.4687 seconds
ov_model on CPU: 0.3058 seconds per image (3.27 FPS)


0.3058163809997495